<h1 align=center> Coursera Capstone Project</h1>
<h2 align='center' style="font-size: 18px"> By: Kyle McLester</h2>
<hr>

<body>This notebook is to demonstrate how to convert addresses into equivalent latitude and longitude values. Also, it will implement the Foursquare API to explore neighborhoods in Toronto, Canada. The neighborhoods will then be clustered using k-means based on the postalcode and borough information.</body>
<hr>

<h3> Import necessary libraries </h3>

In [3]:
import pandas as pd # library to handle data in a vectorized manner

import numpy as np # library for data analysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # library to hand JSON files

from geopy.geocoders import Nominatim # convert an address into lattitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # transform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # library to handle k means clustering algorithm

import folium # library for map rendering

from bs4 import BeautifulSoup # library to help in web scraping

print('Libraries imported successfully')

Libraries imported successfully


<h3> Create Soup Object and Retrieve Data </h3>

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
print('Retrieved HTML data')

Retrieved HTML data


In [8]:
soup = BeautifulSoup(html_data.text, 'html5lib')
print('Created BeautifulSoup object')

Created BeautifulSoup object


In [9]:
tag_object = soup.title
tag_object

<title>List of postal codes of Canada: M - Wikipedia</title>

In [15]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

<h3> Create Pandas DataFrame for table_contents </h3>

In [19]:
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown A PO Boxes25 The Esplande': 'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                                       'EtobicokeNorthWest':'Etobicoke Northwest', 'East YorkEast Toronto': 'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Proecessing Centre':'Mississauga'})

In [23]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [24]:
df.shape

(103, 3)